In [962]:
from sympy import *

In [963]:
from icecream import ic
import pandas as pd

In [964]:
x = symbols('x')
y = symbols("y", cls=Function) # = Function('y')

In [965]:
points = [0.0, 0.2, 0.4, 0.6, 0.8, 1.0]
aEquation = Eq(y(x).diff(x), 2 * y(x) + exp(x) - x)
aInitialCondition = (0, 1/4)  # {y(0): 1 / 4}
bEquation = Eq(y(x).diff(x), x - 1 + (x + 1)*y(x))
bInitialCondition = (0, 0)  # {y(0): 0}

In [966]:
def dsolveExactSolution(equation, initialCondition=(0, 0), pointValues=()):
    initialCondition = {y(initialCondition[0]): initialCondition[1]}
    solution = dsolve(equation, y(x), ics=initialCondition)
    solution = solution.simplify()
    values = [round(solution.rhs.subs(x, p).evalf(10), 5) for p in pointValues]
    return solution, values

In [967]:
a1Solution, a1Values = dsolveExactSolution(aEquation, aInitialCondition, points)
a1Solution, a1Values

(Eq(y(x), x/2 + exp(2*x) - exp(x) + 1/4),
 [0.25000, 0.62042, 1.18372, 2.04800, 3.37749, 5.42077])

In [968]:
b1Solution, b1Values = dsolveExactSolution(bEquation, bInitialCondition, points)
b1Solution, b1Values

(Eq(y(x), (1 + sqrt(2)*sqrt(pi)*exp(1/2)*erf(sqrt(2)/2))*exp(x*(x + 2)/2) - sqrt(2)*sqrt(pi)*exp(x**2/2 + x + 1/2)*erf(sqrt(2)*(x + 1)/2) - 1),
 [0.0, -0.20283, -0.42446, -0.69114, -1.04407, -1.55268])

In [969]:
approximationPrecision = 10

In [970]:
def approximateAnalyticalSolution(equation, initialCondition = (0, 0), Nn = 5, pointValues = ()):
    func = equation.rhs
    x0 = initialCondition[0]
    y0 = initialCondition[1]
    ic(func, x0, y0)
    # Wartosc
    yValues = [y0]
    # Pierwsza pochodna y(1)
    yPrim = func.subs(y(x), y0).subs(x, x0)
    yValues.append(yPrim)

    # Iteracyjnie obliczamy kolejne pochodne y(n)
    for n in range(2, Nn+1):
        func = diff(func, x)
        # Użyj poprzednich wartości pochodnych jako zamienników
        subsDict = {y(x): y0, x: x0}
        for i in range(1, n):
            subsDict[diff(y(x), x, i)] = yValues[i]
        yn = func.subs(subsDict)
        yValues.append(yn)

    taylorSeries = sum(yValues[k] / factorial(k) * ((x - x0) ** k) for k in range(Nn+1))

    approximations = [round(taylorSeries.subs(x, p).evalf(10), 5) for p in pointValues]
    return taylorSeries, approximations


In [971]:
a2Solution, a2Values = approximateAnalyticalSolution(aEquation, aInitialCondition, approximationPrecision, points)
a2Solution, a2Values

ic| func: -x + 2*y(x) + exp(x), x0: 0, y0: 0.25


(0.000281911375661376*x**10 + 0.00140817901234568*x**9 + 0.00632440476190476*x**8 + 0.0251984126984127*x**7 + 0.0875*x**6 + 0.258333333333333*x**5 + 0.625*x**4 + 1.16666666666667*x**3 + 1.5*x**2 + 1.5*x + 0.25,
 [0.25000, 0.62042, 1.18372, 2.04800, 3.37749, 5.42071])

In [972]:
b2Solution, b2Values = approximateAnalyticalSolution(bEquation, bInitialCondition, approximationPrecision, points)
b2Solution, b2Values

ic| func: x + (x + 1)*y(x) - 1, x0: 0, y0: 0


(-71*x**10/90720 - 43*x**9/18144 - 11*x**8/2016 - x**7/63 - x**6/36 - x**5/12 - x**4/12 - x**3/3 - x,
 [0, -0.20283, -0.42446, -0.69114, -1.04403, -1.55226])

In [973]:
def numericalSolution():
    pass

In [974]:
def createTable(tableName, columnHeaders, rowHeaders, tableValues):
    columns = [f"x{i} = {v}" for i, v in enumerate(columnHeaders)]
    table = pd.DataFrame(tableValues, columns=columns, index=rowHeaders)
    table.columns.name = tableName
    return table


In [975]:
aTable = createTable("(a)", points, ["RD", f"RA({approximationPrecision})", "RN"], [a1Values, a2Values, []])
aTable

(a),x0 = 0.0,x1 = 0.2,x2 = 0.4,x3 = 0.6,x4 = 0.8,x5 = 1.0
RD,0.25000,0.62042,1.18372,2.04800,3.37749,5.42077
RA(10),0.25000,0.62042,1.18372,2.04800,3.37749,5.42071
RN,None,None,None,None,None,None


In [976]:
bTable = createTable("(b)", points, ["RD", f"RA({approximationPrecision})", "RN"], [b1Values, b2Values, []])
bTable

(b),x0 = 0.0,x1 = 0.2,x2 = 0.4,x3 = 0.6,x4 = 0.8,x5 = 1.0
RD,0.0,-0.20283,-0.42446,-0.69114,-1.04407,-1.55268
RA(10),0,-0.20283,-0.42446,-0.69114,-1.04403,-1.55226
RN,None,None,None,None,None,None
